# Load libraries and functions

In [424]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [425]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

## Main Code

In [426]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# # occupation = 'shampooers'
# # occupation = 'dredgeOperators'


GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [427]:
# Load the data
onet = pd.read_csv(f'{data_path}/data/onet_occupations_yearly.csv')
onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
onet = onet[onet['year'] == 2023].reset_index(drop=True)

# Get list of tasks
my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
tasks = my_df['task'].unique().tolist()
tasks

['Collect payment for transportation and accommodations from customer.',
 'Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.',
 "Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tour's costs.",
 'Book transportation and hotel reservations, using computer or telephone.',
 'Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.',
 'Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.',
 'Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.',
 'Record and maintain information on clients, vendors, and travel packages.']

<br>

<br>

# 4) GPT Task Partitioning Method

In [428]:
version = 'v0/'
version = 'v1/'

In [429]:
# Set up output file names
partitions_output_filename = f'{occupation_folder}/{version}{occupation}_partitions.csv'
partitioned_DAG_output_filename = f'{occupation_folder}/{version}{occupation}_partitionedGPT_DAG_df.csv'

### Set up questions

In [430]:
if version == 'v0/':
    partitions_relationship_question_text = dedent("""\
                                                Consider {{ occupation }} as an occupation. 
                                                And consider these two partitions of tasks: 
                                                A) {{ partition_A }} 
                                                B) {{ partition_B }}
                                                What is the relationship between these groups of tasks?
                                                Avoid using words like "partition A" and "partition B" in the answer.
                                                Explain the reasoning behind your answer in a couple of sentences.
                                                """)
    partitions_relationship_question_options = {'A first': "Tasks of partition A would be done first",
                                                'B first': "Tasks of partition B would be done first",
                                                'either': "Could be done in either order, but still part of the same sequence",
                                                'neither': "Not part of the same task sequence"
                                                }

    within_partition_task_relationships_question_text = dedent("""\
                                                            Consider {{ occupation }} as an occupation. 
                                                            And consider these two tasks: 
                                                            A) {{ task_A }} 
                                                            B) {{ task_B }}
                                                            What is the relationship between these tasks?
                                                            Avoid using words like "task A" and "task B" in the answer.
                                                            Explain the reasoning behind your answer in a couple of sentences.
                                                            """)
    withinPartition_relationship_question_options = {'A first': "A would be done first",
                                                     'B first': "B would be done first",
                                                     'neither': "Not part of the same task sequence"
                                                    }

    between_partition_task_relationships_question_text = dedent("""\
                                                            Consider {{ occupation }} as an occupation. 
                                                            And consider these two partitions of tasks; partition 1: {{ tasks_partition1 }} and partition 2: {{ tasks_partition2 }}.
                                                            We know that tasks in partition 1 would be done before tasks in partition 2.
                                                            Now consider these two tasks:
                                                            A) {{ task_A }} 
                                                            B) {{ task_B }}
                                                            Knowing that task A is from partition 1 and task B is from partition 2, what is the relationship between these tasks?
                                                            Avoid using words like "task A" and "task B" in the answer.
                                                            Explain the reasoning behind your answer in a couple of sentences.
                                                            """)
    betweenPartition_relationship_question_options = {'A first': "A must be done in order to do B",
                                                      'B first': "A is not required for doing B",
                                                      }

In [431]:
if version == 'v1/':
    partitions_relationship_question_text = dedent("""\
                                                Consider {{ occupation }} as an occupation. 
                                                And consider these two partitions of tasks: 
                                                A) {{ partition_A }} 
                                                B) {{ partition_B }}
                                                Imagine there are two groups of workers, one group are working on partition A tasks and the other are working on partition B tasks.
                                                Do the workers working on partition B tasks need to know the output of the workers working on partition A tasks before getting started?  What about the opposite?
                                                Avoid using words like "partition A" and "partition B" in the answer.
                                                Explain the reasoning behind your answer in a couple of sentences.
                                                """)
    partitions_relationship_question_options = {'A first': "Workers working on partition B tasks need to know the output of workers working on partition A tasks",
                                                'B first': "Workers working on partition A tasks need to know the output of workers working on partition B tasks",
                                                'either': "Either group of workers can start first, but the output of one group is needed by the other group",
                                                'neither': "Neither group of workers needs to know the output of the other group of workers"
                                                }
    
    within_partition_task_relationships_question_text = dedent("""\
                                                            Consider {{ occupation }} as an occupation. 
                                                            And consider these two tasks: 
                                                            A) {{ task_A }} 
                                                            B) {{ task_B }}
                                                            Imagine there are two workers, one working on task A and the other on task B.
                                                            Does the worker working on task B need to know the output of the worker working on task A before getting started? What about the opposite?
                                                            Avoid using words like "task A" and "task B" in the answer.Explain the reasoning behind your answer in a couple of sentences.
                                                            """)
    withinPartition_relationship_question_options = {'A first': "Worker working on task B needs to know the output of worker working on task A",
                                                     'B first': "Worker working on task A needs to know the output of worker working on task B",
                                                     'neither': "Neither worker needs to know the output of the other worker"
                                                     }

    between_partition_task_relationships_question_text = dedent("""\
                                                              Consider {{ occupation }} as an occupation. 
                                                              And consider these two partitions of tasks; partition 1: {{ tasks_partition1 }} and partition 2: {{ tasks_partition2 }}.
                                                              We know that tasks in partition 1 would be done before tasks in partition 2.
                                                              Now consider these two tasks:
                                                              A) {{ task_A }} 
                                                              B) {{ task_B }}
                                                              Imagine there are two workers, one working on task A and the other on task B.
                                                              Does the worker working on task B need to know the output of the worker working on task A before getting started? What about the opposite?
                                                              Avoid using words like "task A" and "task B" in the answer.
                                                              Explain the reasoning behind your answer in a couple of sentences.
                                                              """)
    betweenPartition_relationship_question_options = {'A first': "Worker working on task B needs to know the output of worker working on task A",
                                                      'B first': "Worker working on task A needs to know the output of worker working on task B",
                                                      'neither': "Neither worker needs to know the output of the other worker"
                                                      }

In [432]:
partitions_relationship_question_options_list = list(partitions_relationship_question_options.values())
withinPartition_relationship_question_options_list = list(withinPartition_relationship_question_options.values())
betweenPartition_relationship_question_options_list = list(betweenPartition_relationship_question_options.values())

In [433]:
def partition_relationships(occupation, partitions_dict, question_text, question_options):
    scenarios = [Scenario({"occupation": occupation, "partition_A": A, "partition_B": B}) 
        for A, B in itertools.combinations(partitions_dict.values(), 2)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results



def task_relationships_within_partition(occupation, tasks, question_text, question_options):
    if '"Sink"' in tasks:
        tasks.remove('"Sink"')

    scenarios = [Scenario({"occupation": occupation, "task_A": task_A, "task_B": task_B}) 
        for task_A, task_B in combinations(tasks, 2)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results



def task_relationships_between_partitions(occupation, tasks_partition1, tasks_partition2, question_text, question_options):
    if '"Sink"' in tasks_partition1:
        tasks_partition1.remove('"Sink"')
    if '"Sink"' in tasks_partition2:
        tasks_partition2.remove('"Sink"')
    

    scenarios = [Scenario({"occupation": occupation, 
                           "tasks_partition1": tasks_partition1, "tasks_partition2": tasks_partition2,
                           "task_A": task_A, "task_B": task_B}) 
        for task_A, task_B in itertools.product(tasks_partition1, tasks_partition2)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results



## Break down the DAG into multiple minimally-connected subgraphs

In [434]:
task_partitioning_question_text =  dedent("""\
                                          Consider {{ occupation }} as an occupation. 
                                          And consider these tasks: {{ tasks }}.
                                          Can these tasks be partitioned into separate, minimally connected groups of tasks?
                                          If so, give the number of groups and list tasks in each group. 
                                          Avoid using \n in the answer, and list groups in the following format: Group x: ['task1', 'task2', 'task3'].
                                          """)

def partition_tasks(occupation, tasks, question_text):
    if '"Sink"' in tasks:
        tasks.remove('"Sink"')

    scenarios = [Scenario({"occupation": occupation, "tasks": tasks})]

    q = QuestionFreeText(
        question_name = "partition",
        question_text = question_text
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results

results = partition_tasks(GPT_input_occupation, tasks, task_partitioning_question_text)
#results.print()
partition_tasks_output_str = results.select("answer.partition").to_pandas().iloc[0,0]

Output()

#### Group tasks into smaller partitions

In [435]:
# Find all "Group x" occurrences in LLM output
groups = re.findall(r'Group \d+', partition_tasks_output_str)

# Split the text at each "Group x"
parts = re.split(r'(Group \d+:)', partition_tasks_output_str)

# Initialize a dictionary to hold the group texts
partitions_dict = {}

# Iterate through the parts and store the texts in the dictionary
for i in range(1, len(parts), 2):
    group_name = parts[i].strip(': ')
    group_number = int(re.search(r'\d+', group_name).group())
    group_text = parts[i+1].strip().rstrip('.,')
    
    # Convert the string representation of the list to an actual list
    partitions_dict[group_number] = group_text

# Save partitions
partitions_df = pd.DataFrame.from_dict(partitions_dict, orient='index')
partitions_df.to_csv(partitions_output_filename, index=False, header=False) 

## Determine relation of partitions

In [436]:
# Compare pair of partitions
results = partition_relationships(GPT_input_occupation, partitions_dict, partitions_relationship_question_text, partitions_relationship_question_options_list)
#results.print()
partitions_ordering_df = results.select("partition_A", "partition_B", "ordering", "ordering_comment").to_pandas()

Output()

In [437]:
partitions_ordering_df

,answer.ordering,comment.ordering_comment,scenario.partition_A,scenario.partition_B
0,Workers working on partition A tasks need to k...,Workers working on partition A tasks need to k...,['Collect payment for transportation and accom...,['Converse with customer to determine destinat...
1,Neither group of workers needs to know the out...,The tasks in each group are independent of eac...,['Collect payment for transportation and accom...,['Provide customer with brochures and publicat...
2,Workers working on partition B tasks need to k...,Workers working on partition B tasks need to k...,['Converse with customer to determine destinat...,['Provide customer with brochures and publicat...


In [438]:
# Swap columns so that all partitions in first column are done earlier
mask = partitions_ordering_df['answer.ordering'] == partitions_relationship_question_options['B first']
partitions_ordering_df.loc[mask, ['scenario.partition_A', 'scenario.partition_B']] = partitions_ordering_df.loc[mask, ['scenario.partition_B', 'scenario.partition_A']].values
partitions_ordering_df.loc[mask, 'answer.ordering'] = partitions_relationship_question_options['A first']
partitions_ordering_df = partitions_ordering_df[partitions_ordering_df['answer.ordering'] == partitions_relationship_question_options['A first']]
partitions_ordering_df

,answer.ordering,comment.ordering_comment,scenario.partition_A,scenario.partition_B
0,Workers working on partition B tasks need to k...,Workers working on partition A tasks need to k...,['Converse with customer to determine destinat...,['Collect payment for transportation and accom...
2,Workers working on partition B tasks need to k...,Workers working on partition B tasks need to k...,['Converse with customer to determine destinat...,['Provide customer with brochures and publicat...


In [439]:
# Add group numbers to data frame
aux_dict = {v: k for k, v in partitions_dict.items()}
partitions_ordering_df['partition_A_groupNum'] = partitions_ordering_df['scenario.partition_A'].map(aux_dict)
partitions_ordering_df['partition_B_groupNum'] = partitions_ordering_df['scenario.partition_B'].map(aux_dict)
partitions_ordering_df

,answer.ordering,comment.ordering_comment,scenario.partition_A,scenario.partition_B,partition_A_groupNum,partition_B_groupNum
0,Workers working on partition B tasks need to k...,Workers working on partition A tasks need to k...,['Converse with customer to determine destinat...,['Collect payment for transportation and accom...,2,1
2,Workers working on partition B tasks need to k...,Workers working on partition B tasks need to k...,['Converse with customer to determine destinat...,['Provide customer with brochures and publicat...,2,3


## Compare pair of tasks within each partition

In [440]:
# Function to handle apastrophes and commas in the list string
def clean_list_string(s):
    # Escape the apostrophe in specific problematic cases
    s = re.sub(r"(?<!\\)'s costs", r"\\'s costs", s)
    return s


task_relationships_within_partition_df = pd.DataFrame()
for key, value in partitions_dict.items():
    # Get list of tasks in the partition
    my_partition_tasks = clean_list_string(value)
    my_partition_tasks = ast.literal_eval(my_partition_tasks)
    if len(my_partition_tasks) < 2:
        continue

    # Run the function
    results = task_relationships_within_partition(GPT_input_occupation, my_partition_tasks, within_partition_task_relationships_question_text, withinPartition_relationship_question_options_list)
    aux_df = results.select("task_A", "task_B", "ordering", "comment.ordering_comment").to_pandas()
    aux_df['partition'] = key

    # Add to data frame
    task_relationships_within_partition_df = pd.concat([task_relationships_within_partition_df, aux_df], ignore_index=True)

Output()

Output()

Output()

In [441]:
# Swap columns so that all tasks in first column are done earlier
mask = task_relationships_within_partition_df['answer.ordering'] == withinPartition_relationship_question_options['B first']
task_relationships_within_partition_df.loc[mask, ['scenario.task_A', 'scenario.task_B']] = task_relationships_within_partition_df.loc[mask, ['scenario.task_B', 'scenario.task_A']].values
task_relationships_within_partition_df.loc[mask, 'answer.ordering'] = withinPartition_relationship_question_options['A first']

## Compare pair of tasks between partitions

In [442]:
task_relationships_between_partitions_df = pd.DataFrame()
for (key1, value1), (key2, value2) in itertools.combinations(partitions_dict.items(), 2):
    # determine which partition is done first
    if len(partitions_ordering_df[(partitions_ordering_df['partition_A_groupNum'] == key1) & (partitions_ordering_df['partition_B_groupNum'] == key2)]) > 0:
        first_partition = key1
        second_partition = key2
    elif len(partitions_ordering_df[(partitions_ordering_df['partition_A_groupNum'] == key2) & (partitions_ordering_df['partition_B_groupNum'] == key1)]) > 0:
        first_partition = key2
        second_partition = key1
    else:
        continue

    # Get list of tasks in the partition
    tasks_partition1 = ast.literal_eval(clean_list_string(value1))
    tasks_partition2 = ast.literal_eval(clean_list_string(value2))
    
    # Run the function
    results = task_relationships_between_partitions(GPT_input_occupation, tasks_partition1, tasks_partition2, between_partition_task_relationships_question_text, betweenPartition_relationship_question_options_list)
    aux_df = results.select("task_A", "task_B", "ordering", "comment.ordering_comment").to_pandas()
    
    # Add to data frame
    task_relationships_between_partitions_df = pd.concat([task_relationships_between_partitions_df, aux_df], ignore_index=True)

Output()

Output()

In [443]:
# Swap columns so that all tasks in first column are done earlier
mask = task_relationships_between_partitions_df['answer.ordering'] == betweenPartition_relationship_question_options['B first']
task_relationships_between_partitions_df.loc[mask, ['scenario.task_A', 'scenario.task_B']] = task_relationships_between_partitions_df.loc[mask, ['scenario.task_B', 'scenario.task_A']].values
task_relationships_between_partitions_df.loc[mask, 'answer.ordering'] = betweenPartition_relationship_question_options['A first']

In [444]:
# Get edges from within and between partitions data frames
between_edges = task_relationships_between_partitions_df[task_relationships_between_partitions_df['answer.ordering'] == betweenPartition_relationship_question_options['A first']]
between_edges = between_edges[['scenario.task_A', 'scenario.task_B', 'comment.ordering_comment']]

within_edges = task_relationships_within_partition_df[task_relationships_within_partition_df['answer.ordering'] == withinPartition_relationship_question_options['A first']]
within_edges = within_edges[['scenario.task_A', 'scenario.task_B', 'comment.ordering_comment']]


# Combine edges from within and between partitions
partitions_DAG_df = pd.concat([within_edges, between_edges], ignore_index=True)

# Change column names
partitions_DAG_df = partitions_DAG_df.rename(columns={'scenario.task_A': 'source', 
                                                    'scenario.task_B': 'target', 
                                                    'comment.ordering_comment': 'comment'})

# Save output
partitions_DAG_df.to_csv(partitioned_DAG_output_filename, index=False)